In [ ]:
""" Converts multiline animation sprite to a single line srpite. Generates the animation sequence for the scene file. """
import numpy as np
from PIL import Image
from pathlib import Path
base_dir = Path("../Assets/World/Buildings/Agricultural/CattleRun/Sprites")
src_image = Image.open(base_dir / "CattleRun_work_45.png")
tile_size = np.array([192, 192])
cols_rows = np.array([15, 10])
width_height = (tile_size[0]*cols_rows[0]*cols_rows[1], tile_size[1]) # one wide image
dst_image = Image.new("RGBA", tuple(width_height))
for i in range(0, 10):
  crop = np.array([0,0,tile_size[0]*cols_rows[0],tile_size[1]])
  crop += np.array([0,tile_size[1]*i,0,tile_size[1]*i])
  # display(crop)
  dst_image.paste(src_image.crop(tuple(crop)),(i*tile_size[0]*cols_rows[0],0))
dst_image.save(base_dir / "CattleRun_work_flat_45.png")
display(dst_image)

for i in range(cols_rows[0]*cols_rows[1]):
  print(f"0:0/animation_frame_{i}/duration = 1.0")

In [ ]:
# [ext_resource type="Texture2D" uid="uid://bvbq4ud7b5cx6" path="res://Assets/World/Buildings/Agricultural/CattleRun/Sprites/CattleRun_work_45.png" id="1_nncli"]
# generate atlas textures:
for y in range(0, 15):
	for x in range(0, 20):
		src_resource_id = "1_nncli"
		x_start = x*192
		x_end = (x+1)*192
		y_start = y*192
		y_end = (y+1)*192
		print(
f"""
[sub_resource type="AtlasTexture" id="AtlasTexture_{x:02}{y:02}"]
atlas = ExtResource("1_nncli")
region = Rect2({x_start}, {y_start}, {x_end}, {y_end})""")

for y in range(0, 15):
	for x in range(0, 20):
		print(
f"""{{
"duration": 1.0,
"texture": SubResource("AtlasTexture_{x:02}{y:02}")
}},"""
		)


In [ ]:
import os
# godot_parser - godot 3 file format only? doesn't support godot 4 animations
from godot_parser import *
from pathlib import Path

# Create a new TSCN file
file_content = Path("../Assets/World/Buildings/Agricultural/CattleRun/CattleRun2D.tscn").read_text()
scene = GDScene.parse(file_content)
scene.write(Path("../Assets/World/Buildings/Agricultural/CattleRun/CattleRun2D.2.tscn"))

In [ ]:
import numpy as np
from PIL import Image, ImageDraw
import math

def detect_sprite_size(image_path, min_percentage_filled=0.1):
  image = Image.open(image_path).convert("RGBA")
  img = np.array(image) # img.shape == (height, width, 4)
  img_alpha = img[:, :, 3] # img_alpha.shape == (height, width)
  img_height, img_width = img_alpha.shape

  # possible_widths = [w for w in range(64, img_width+1, 64)]
  # possible_heights = [h for h in range(32, img_height+1, 32)]
  possible_widths = [w for w in range(64, min(64*8, img_width)+1, 64)]
  possible_heights = [h for h in range(32, min(32*8, img_height)+1, 32)]
  # possible_widths = [128]
  # possible_heights = [128]
  width_stats = []
  for frame_width in possible_widths:
    aligned_width = frame_width * math.floor(img_width/frame_width)
    img_columns = np.array(np.split(img_alpha[:,0:aligned_width], aligned_width/frame_width, axis=1))
    # img_columns.shape == (frame_count, img_height, frame_width)
    while (not np.any(img_columns[-1])):
      img_columns = img_columns[:-1] # trim trailing zero columns
    leftover_chunk = img_alpha[:, aligned_width:img_width]
    if np.count_nonzero(leftover_chunk) > 0:
      continue # leftover contains some pixels, skip this width
    
    # sum all frames and find the left+right border filled count:
    left_right_border_filled_count = np.count_nonzero(img_columns[:,:,0]) + np.count_nonzero(img_columns[:,:,-1])

    per_frame_filled = np.count_nonzero(img_columns, axis=(1,2)) # shape == (frame_count,)
    min_pixels_count = img_columns.shape[1] * img_columns.shape[2] * 0.05
    if (np.any(per_frame_filled < min_pixels_count)):
      continue # some frames have less than 5% filled pixels, skip this width
    width_stats.append({
      "border_filled": left_right_border_filled_count / (img_columns.shape[0]*img_columns.shape[1]*2),
      "frame_width": frame_width,
      "cols_count": img_columns.shape[0],
    })
    

  # sometimes there are multiple heights with the similar border_filled value, choose the one with smallest frame_height
  # while being withing threshold of 1.5x from min_val
  min_val = min(width_stats, key=lambda x: x['border_filled'])
  best_width = min([hs for hs in width_stats if hs['border_filled'] <= min_val['border_filled']+0.03], key=lambda x: x['frame_width'])
  # print("best_width:", best_width)

  # check heights:
  height_stats = []
  for frame_height in possible_heights:
    aligned_height = frame_height * math.floor(img_height/frame_height)
    img_rows = np.array(np.split(img_alpha[0:aligned_height,:], aligned_height/frame_height, axis=0))
    # img_rows.shape == (frame_count, frame_height, img_width)
    # display(PIL.Image.fromarray(img_rows[0]))
    while (not np.any(img_rows[-1])):
      img_rows = img_rows[:-1] # trim trailing zero rows
    leftover_chunk = img_alpha[aligned_height:img_height, :]
    if np.count_nonzero(leftover_chunk) > 0:
      continue # leftover contains some pixels, skip this height
    
    # sum all frames and find the left+right border filled count:
    top_bottom_border_filled_count = np.count_nonzero(img_rows[:,0,:]) + np.count_nonzero(img_rows[:,-1,:])

    per_frame_filled = np.count_nonzero(img_rows, axis=(1,2)) # shape == (frame_count,)
    min_pixels_count = img_rows.shape[1] * img_rows.shape[2] * 0.05
    if (np.any(per_frame_filled < min_pixels_count)):
      continue # some frames have less than 5% filled pixels, skip this height
    height_stats.append({
      "border_filled": top_bottom_border_filled_count / (img_rows.shape[0]*img_rows.shape[2]*2),
      "frame_height": frame_height,
      "rows_count": img_rows.shape[0],
    })

  # sometimes there are multiple heights with the similar border_filled value, choose the one with smallest frame_height
  # while being withing threshold from min_val
  min_val = min(height_stats, key=lambda x: x['border_filled'])
  best_height = min([hs for hs in height_stats if hs['border_filled'] <= min_val['border_filled'] + 0.03], key=lambda x: x['frame_height'])
  # print("best_height:", best_height)
  return best_width['frame_width'], best_height['frame_height'], best_width['cols_count'], best_height['rows_count']


from pathlib import Path
building_path = Path(r"s:\src\unknown-horizons-godot-port\Assets\World\Buildings")
building_paths = building_path.glob(r"**\*.png")
# building_paths = building_path.glob(r"**\CattleRun_work_135.png")
# building_paths = building_path.glob(r"**\CattleRun_idle.png")
# building_paths = building_path.glob(r"**\MainSquareStone_idle.png")
# building_paths = building_path.glob(r"**\CharcoalBurning_work_anim*.png")
# building_paths = building_path.glob(r"**\Bath_idle.png")
for image_path in building_paths:
  print("processing:", image_path)
  if ("LumberjackHut_overlay_" in str(image_path) or 
      "Buildings\\Streets" in str(image_path) or
      "Buildings\\Wall" in str(image_path) or
      "Buildings\\Lumberjack" in str(image_path) or
      "Buildings\\TestBlank" in str(image_path) or
      "Buildings\\Toolmaker" in str(image_path) or
      "Buildings\\Distillery" in str(image_path) or
      "Buildings\\CharcoalBurning" in str(image_path)
      ):
    print("skipped")
    continue
  frame_width, frame_height, cols, rows = detect_sprite_size(image_path)
  img = Image.open(image_path)
  img = img.crop((0, 0, frame_width, frame_height))
  draw = ImageDraw.Draw(img)
  draw.rectangle((0, 0, frame_width-1, frame_height-1), outline=(255,255,255,255))
  print(f"Detected Sprite Size: {frame_width}x{frame_height}, {cols}x{rows} for `{image_path.relative_to(building_path)}`")
  display(img)
  # break


In [ ]:
from pathlib import Path, PurePosixPath
from jinja2 import Environment, FileSystemLoader
import re

building_gd_template = """
extends ProductionBuilding2D

class_name {{ scene_name }}2D

func _ready():
  self.setup_building()
  self.should_produce = true
{# newline at end #}
"""

building_tscn_template = """
[gd_scene load_steps=7 format=3]

[ext_resource type="PackedScene" uid="uid://c14jc7hm1oowb" path="res://Assets/World/Buildings/ProductionBuilding2D.tscn" id="1_ProductionBuilding2D"]
[ext_resource type="Script" path="res://{{ script_path }}" id="2_script"]
[ext_resource type="Resource" path="res://{{ building_data_path }}" id="3_building_data"]
[ext_resource type="SpriteFrames" path="res://{{ sprite_frames_res_path }}" id="4_animated_sprite"]
[ext_resource type="Texture2D" uid="uid://brmfkn8sfum3n" path="res://Assets/UI/Icons/Resources/32/005.png" id="5_tooltip_image"]
[ext_resource type="PackedScene" uid="uid://x1upwhg1f71a" path="res://Assets/World/Behavior/Selectable/Selectable.tscn" id="6_selectable"]
{%- if tab_widget_path != None: %}
[ext_resource type="PackedScene" path="res://{{ tab_widget_path }}" id="id_info_tab_widget"]
{% endif %}

[node name="{{ scene_name }}2D" instance=ExtResource("1_ProductionBuilding2D")]
script = ExtResource("2_script")
building_data = ExtResource("3_building_data")
info_tab_widget = {{ 'ExtResource("id_info_tab_widget")' if tab_widget_path != None else 'null' }} 
building_size = Vector2i({{ building_size }})
built_tile_map_tile_id = {{ built_tile_map_tile_id }}

[node name="BuildingSprite" type="AnimatedSprite2D" parent="." index="0"]
sprite_frames = ExtResource("4_animated_sprite")
offset = Vector2(0, -16)

[node name="ItemProducedTooltip" parent="." index="2"]
texture = ExtResource("5_tooltip_image")
amount = "0"

[node name="Selectable" parent="." index="3" instance=ExtResource("6_selectable")]

[node name="ReferenceRect" type="ReferenceRect" parent="." index="4"]
offset_left = -32.0
offset_top = -16.0
offset_right = 32.0
offset_bottom = 16.0
mouse_filter = 2
"""

building_data_tres_template = """
[gd_resource type="Resource" script_class="ProductionBuildingData" load_steps=5 format=3]

[ext_resource type="Resource" uid="uid://cadptbki7hpwf" path="res://Assets/World/Data/ItemData/Food.tres" id="id_output_tres"]
[ext_resource type="Resource" uid="uid://c5xn4hbcdts6c" path="res://Assets/World/Data/ItemData/Timber.tres" id="id_cost_res"]
[ext_resource type="Script" path="res://Assets/World/Data/BuildingData/ProductionBuildingData.gd" id="id_script"]

[resource]
script = ExtResource("id_script")
max_storage_capacity = 10
processing_time = 3
load_or_unload_time = 2.0
output_product = ExtResource("id_output_tres")
input_products = {}
game_name = "{{ game_name }}"
cost = {
ExtResource("id_cost_res"): 1
}
"""

sprite_frames_tres_template = """
[gd_resource type="SpriteFrames" load_steps={{ images|length + 1 }} format=3]
{% for image in images %}
[ext_resource type="Texture2D" path="res://{{ image.image_path }}" id="{{ image.image_path.name }}"]
{%- endfor %}
{% for image in images -%}
{% for row in range(image.rows) -%}
{% for col in range(image.cols) %}
[sub_resource type="AtlasTexture" id="AtlasTexture_{{ image.animation_name }}_{{ row }}_{{ col }}"]
atlas = ExtResource("{{ image.image_path.name }}")
region = Rect2({{ col * image.frame_width }}, {{ row * image.frame_height }}, {{ image.frame_width }}, {{ image.frame_height }})
{% endfor %}
{%- endfor %}
{%- endfor %}
[resource]
animations = [
  {%- for image in images %}
    {%- if image.animation_name.endswith("idle") -%}
    {# special case for `idle`. `idle` spritesheets contain all four side images in a 2x2 row #}
; `idle_anim`:
    {%- for row in range(image.rows) %}
    {%- for col in range(image.cols) %}
{
"frames": [
{
"duration": 1.0,
"texture": SubResource("AtlasTexture_{{ image.animation_name }}_{{ row }}_{{ col }}")
},
],
"loop": true,
"name": &"{{ build_idle_name(image.animation_name, ["idle_045", "idle_135", "idle_225", "idle_315"][row*2+col]) }}",
"speed": 5.0
},
    {%- endfor %}
    {%- endfor %}{# {% for row in range(image.rows) %} #}
    {%- elif image.animation_name == "work_anim" -%}
    {# special case for work_anim. work_anim spritesheets contain all four animations combined, split them in individual animations #}
  ; `work_anim`:
    {%- for row in range(image.rows) %}
{
"frames": [
{%- for col in range(image.cols) %}
{
"duration": 1.0,
"texture": SubResource("AtlasTexture_{{ image.animation_name }}_{{ row }}_{{ col }}")
},
{%- endfor %}
],
"loop": true,
"name": &"{{ ["work_045", "work_135", "work_225", "work_315"][row] }}",
"speed": 5.0
},
    {%- endfor -%}{# {% for row in range(image.rows) %} #}
    {%- else -%}{# switch image.animation_name #}
{
"frames": [
      {%- for row in range(image.rows) %}
      {%- for col in range(image.cols) %}
{
  "duration": 1.0,
  "texture": SubResource("AtlasTexture_{{ image.animation_name }}_{{ row }}_{{ col }}")
},
      {%- endfor %}
      {%- endfor %}
],
"loop": true,
"name": &"{{ image.animation_name }}",
"speed": 5.0
},
    {%- endif -%}{# {% if image.animation_name != "work_anim" %} -#}
  {%- endfor %}
]
"""

class SpriteImage:
  def __init__(self, image_path: str, frame_width: int, frame_height: int, cols: int, rows: int, animation_name: str, uid: str):
    self.image_path = image_path
    self.frame_width = frame_width
    self.frame_height = frame_height
    self.cols = cols
    self.rows = rows
    self.animation_name = animation_name
  def __repr__(self):
    return (f"SpriteImage(image_path={self.image_path}, frame_width={self.frame_width}, frame_height={self.frame_height}, " + 
           f"cols={self.cols}, rows={self.rows}, animation_name={self.animation_name}")

# base_path = PurePosixPath("Assets/World/Buildings/Agricultural/CattleRun")
# base_os_path = Path("s:/src/unknown-horizons-godot-port") / base_path
# images = [
#   SpriteImage(base_path / "Sprites/CattleRun_work_45.png", 192, 192, 15, 10, "work_045", "bvbq4ud7b5cx6"),
#   SpriteImage(base_path / "Sprites/CattleRun_work_135.png", 192, 192, 15, 10, "work_135", "cho3yy7cxqnwd"),
#   SpriteImage(base_path / "Sprites/CattleRun_work_225.png", 192, 192, 15, 10, "work_225", ""),
#   SpriteImage(base_path / "Sprites/CattleRun_work_315.png", 192, 192, 15, 10, "work_315", ""),
# ]
# write_template(base_os_path / "Sprites/CattleRun2D.tres", sprite_frames_tres_template, {
#    "images": images,
#    "uid" = ""
# })

def get_tileset_scenes_dict(tileset_path: Path):
  content = tileset_path.read_text()
  # match section
  ##[sub_resource type="TileSetScenesCollectionSource"...
  ##resource_name = "Buildings"
  ##...
  ##[sub_resource...
  rr = re.compile(r'(?P<header>\[sub_resource type=\"TileSetScenesCollectionSource\".*\n'+
                  r'resource_name = \"Buildings\"\n)'+
                  r'(?P<scenes>(?:.+\n)*?)'+
                  r'(?=\n*\[sub_resource)') # until next sub_resource
  section_match = [m for m in rr.finditer(content)][0]

  # match following strings:
  ##scenes/.../scene = ExtResource("...\.tscn")
  r = re.compile(r'scenes/(?P<tile_idx>\d+)/scene\s=\sExtResource\("(?P<scene_id>\S+\.tscn)"\)')
  tileset_scenes_matches = [m for m in r.finditer(content)]
  assert(all([sm.start() >= section_match.start() for sm in tileset_scenes_matches]))
  assert(all([sm.end() <= section_match.end() for sm in tileset_scenes_matches]))

  tileset_scenes_dict = {m.groupdict()["scene_id"]: int(m.groupdict()["tile_idx"]) for m in tileset_scenes_matches}

  content_pre = content[:section_match.start()] + section_match["header"]
  content_post = content[section_match.end():]

  return tileset_scenes_dict, content_pre, content_post

def write_template(output_path: Path, template_str: str, args: dict):
  print(f"Writing template: {output_path}")
  env = Environment(loader=FileSystemLoader("."))
  template = env.from_string(template_str.strip())
  rendered_content = template.render(args)
  output_path.write_text(rendered_content)

project_path = Path("s:/src/unknown-horizons-godot-port")
# buildings_path = project_path / "Assets/World/Buildings/Agricultural"
buildings_path = project_path / "Assets/World/Buildings"
folders_to_check = [path for path in buildings_path.glob("**")] # includes Agricultural and others subdir
folders_to_check = [folder for folder in folders_to_check if folder.is_dir() and (folder / "Sprites").is_dir()]
# folders_to_check = [folder for folder in folders_to_check if folder.stem == "Bakery"] # debug
# folders_to_check = [folder for folder in folders_to_check if "WoodenTower" in folder.stem] # debug
for folder in folders_to_check:
  print(f"Processing: {folder}")

  if ("LumberjackHut_overlay_" in str(folder) or 
      "Buildings\\Streets" in str(folder) or
      "Buildings\\Wall" in str(folder) or
      "Buildings\\Residential" in str(folder) or 
      "Buildings\\Lumberjack" in str(folder) or
      "Buildings\\TestBlank" in str(folder) or
      "Buildings\\Toolmaker" in str(folder) or
      "Buildings\\Distillery" in str(folder) or
      "Buildings\\CharcoalBurning" in str(folder)
      ):
    print("skipped")
    continue

  img_paths = list(folder.glob("**/*.png"))
  if (set([p.parent.name for p in img_paths]) != set(["Sprites"])):
    raise Exception("Unexpected folder structure. Expected images to be in folder \"Sprites\"")
  if (len(set(p.parent.parent for p in img_paths)) != 1):
    raise Exception("Unexpected folder structure. Expected all images to be in the same scene folder")
  scene_name = img_paths[0].parent.parent.name
  print(f"Scene name: {scene_name}")

  base_posix_path = PurePosixPath(folder.relative_to(project_path))

  def get_sprite_image(img_path: Path, scene_name: str):
    proj_relative_path = PurePosixPath(img_path.relative_to(project_path))
    animation_name = proj_relative_path.stem.removeprefix(scene_name).removeprefix("_")
    animation_name = animation_name.replace("_45", "_045")
    frame_width, frame_height, cols, rows = detect_sprite_size(img_path)
    sprite_image = SpriteImage(proj_relative_path, frame_width, frame_height, cols, rows, animation_name, None)
    return sprite_image

  def build_idle_name(animation_name: str, frame_name: str):
    base_name = animation_name.removesuffix("idle").removesuffix("_")
    res =  f"{base_name}{"_" if base_name else ""}{frame_name}"
    return res

  images: list[SpriteImage] = [get_sprite_image(p, scene_name) for p in img_paths]
  sprites_tres_filepath = folder / f"Sprites/{scene_name}2D.tres"
  display(images)
  print(f"Writing sprites_tres_filepath: {sprites_tres_filepath}")
  write_template(sprites_tres_filepath, sprite_frames_tres_template, {
    "images": images,
    "build_idle_name": build_idle_name,
  })

  scene_path = folder / f"{scene_name}2D.tscn"
  script_path = base_posix_path / f"{scene_path.stem}.gd"
  building_data_path = PurePosixPath("Assets/World/Data/BuildingData") / f"{scene_path.stem}Data.tres"
  sprite_frames_res_path = PurePosixPath(sprites_tres_filepath.relative_to(project_path))
  
  tab_widget_scenes = list(project_path.glob(f"**/{scene_name}TabWidget.tscn", case_sensitive=False))
  if len(tab_widget_scenes) > 1:
    raise Exception(f"Expected to find exactly one tab widget scene named {scene_name}TabWidget.tscn, but found {len(tab_widget_scenes)}")
  elif len(tab_widget_scenes) == 0:
    # fields do not have their own tab widgets:
    if scene_name not in ["Alvearies", "CattleRun", "CocoaField", "CornField", "Herbary", "HopField", "Pasture", "PotatoField", "SpiceField", "SugarField", "TobaccoField", "Vineyard", ]:
      display(Exception(f"Could not find tab widget scene named {scene_name}TabWidget.tscn"))
    tab_widget_path = None
  else:
    tab_widget_path = PurePosixPath(tab_widget_scenes[0].relative_to(project_path))
  
  print(f"script_path: res://{script_path}")
  print(f"building_data_path: res://{building_data_path}")
  print(f"sprite_frames_res_path: res://{sprite_frames_res_path}")
  print(f"tab_widget_path: res://{tab_widget_path}")

  # lookup the building_tile (tileset_id) in the builttileset.tres:
  tileset_path = project_path / "Assets/World/Tilemaps/BuiltTileMap/BuiltTileSet.tres"
  tileset_scenes_dict, built_tileset_content_pre, built_tileset_content_post = get_tileset_scenes_dict(tileset_path)
  tileset_scene_id = scene_path.name # Bakery2D.tscn
  built_tile_map_tile_id = tileset_scenes_dict.get(tileset_scene_id)
  built_tile_map_tile_id_existed = built_tile_map_tile_id is not None
  if not built_tile_map_tile_id_existed:
    used_ids = set(tileset_scenes_dict.values())
    built_tile_map_tile_id = 1
    while built_tile_map_tile_id in used_ids:
      built_tile_map_tile_id += 1
    tileset_scenes_dict[tileset_scene_id] = built_tile_map_tile_id


  write_template(project_path / script_path, building_gd_template, {
    "scene_name": scene_name,
  })

  write_template(project_path / building_data_path, building_data_tres_template, {
    "game_name": scene_name.lower(),
  })

  write_template(scene_path, building_tscn_template, {
    "script_path": script_path,
    "building_data_path": building_data_path,
    "sprite_frames_res_path": sprite_frames_res_path,
    "tab_widget_path": tab_widget_path,
    "building_size": f"{images[0].frame_width//64}, {images[0].frame_width//64}", # TODO: adjust height manually
    "scene_name": scene_name,
    "built_tile_map_tile_id": built_tile_map_tile_id,
  })

  # update the tileset precontent to include the resource:
  #[ext_resource type="PackedScene" path="res://Assets/World/Buildings/Bakery/Bakery2D.tscn" id="Bakery2D.tscn"]
  re_ext_resource = re.compile(r'\[ext_resource type="PackedScene".*path="(?P<path>res://Assets/World/Buildings/.*?)".*\n')
  ext_resources_matches = [ext_resource for ext_resource in re_ext_resource.finditer(built_tileset_content_pre)]
  ext_resources_paths = {m.groupdict()["path"]: m for m in ext_resources_matches}
  scene_res_path = PurePosixPath(scene_path.relative_to(project_path))
  scene_match = ext_resources_paths.get(f"res://{scene_res_path}")
  if scene_match is None:
    last_ext_res_end = ext_resources_matches[-1].end()
    entry = f'[ext_resource type="PackedScene" path="res://{scene_res_path}" id="{scene_res_path.name}"]\n'
    built_tileset_content_pre = built_tileset_content_pre[:last_ext_res_end] + entry + built_tileset_content_pre[last_ext_res_end:]

  
  if not built_tile_map_tile_id_existed or scene_match is None:
    # update the tileset
    scenes_sorted = sorted((tile_id, tileset_scene_id) for (tileset_scene_id, tile_id) in tileset_scenes_dict.items())
    scenes_str = "".join([f'scenes/{tile_id}/scene = ExtResource("{tileset_scene_id}")\n' for (tile_id, tileset_scene_id) in scenes_sorted])
    new_content = built_tileset_content_pre + scenes_str + built_tileset_content_post
    # (Path(str(tileset_path).replace(".tres", ".new.tres"))).write_text(new_content)
    tileset_path.write_text(new_content)
    pass

  